Il progetto trae ispirazione dalle tecniche sviluppate da Pangeanic e dal suo prodotto commerciale. Sebbene non ci sia l'intenzione di competere direttamente con Pangeanic, il loro approccio ha fornito preziosi spunti e chiarezza sulle possibili soluzioni per il problema del "NER Tagging e Anonimizzazione".

[Link alle soluzioni di Pangeanic per l'anonimizzazione](https://pangeanic.com/it/soluzioni-nlp/anonimizzazione)

al momento il codice gestisce i tag PER , ORG , LOC e MISC.
le entità di tipo PER ed ORG vengono riconosciute, accorpate e sostituite con un randomico iperinomio, viene gestito inoltre, tramite il tag nel testo, una molteplice occorrenza dell'entità che viene indicata con il numero assegnato.
Per quel che riguarda le LOC, vengono sostituiti dal tag generico [Location], sempre numerato, ma è possibile aggiungere più dettagli.
Il MISC invece viene solo segnalato al redattore, in quanto un tag generico non ha uno standard di gestione.

In [38]:
import pandas as pd
import numpy as np
import pickle 

def read_tagging(file_name):
    path="../data/"+"it"+"/tagging/"+file_name+".conllu"
    data = pd.read_csv (path, sep = '\t',quoting=3, names=["POSITION","WORD","TAG"])
    return data

In [39]:
import pandas as pd
import random

def anonymize_document(ner_df):

    # Conversione di tutte le parole a stringhe
    ner_df['WORD'] = ner_df['WORD'].astype(str)
    
    # Dizionario che tiene traccia del numero di pseudonimi già assegnati per ogni tipo di entità
    pseudonym_counters = {
        'PER': {},  # Contatore per le entità di tipo PERSON
        'ORG': {},  # Contatore per le entità di tipo ORGANIZATION
        'LOC': {},  # Contatore per le entità di tipo LOCATION
        'MISC': 1   # Contatore per le entità di tipo MISC (inizia da 1)
    }

    # Dizionario che mappa gli pseudonimi già assegnati a ciascuna entità
    pseudonyms = {}

    # Dizionario di sinonimi o iperonimie generici per PERSON e ORGANIZATION
    generic_synonyms_or_hypernyms = {
        'PER': ['soggetto', 'individuo', 'persona', 'cittadino'],
        'ORG': ['compagnia', 'azienda', 'entità', 'organizzazione']
    }

    # Funzione per ottenere o generare un pseudonimo per un'entità di tipo PERSON, ORGANIZATION o LOCATION
    def get_pseudonym(entity_type, entity):
        if entity_type in ['PER', 'ORG', 'LOC']:  # Verifica se l'entità è di uno dei tipi gestiti
            if entity in pseudonyms:  # Verifica se l'entità ha già un pseudonimo assegnato
                return pseudonyms[entity]  # Restituisce l'pseudonimo già assegnato
            else:
                # Se l'entità non ha ancora un pseudonimo, genera uno nuovo
                if entity not in pseudonym_counters[entity_type]:
                    # Assegna un numero incrementale come identificativo dell'entità
                    pseudonym_counters[entity_type][entity] = len(pseudonym_counters[entity_type]) + 1
                # Costruisce l'pseudonimo con il formato corretto
                pseudonym = f"[{entity_type.capitalize()}{pseudonym_counters[entity_type][entity]}]"
                # Se è disponibile un elenco di sinonimi generici per il tipo di entità, aggiunge un sinonimo casuale
                if entity_type in generic_synonyms_or_hypernyms:
                    pseudonym += f" {random.choice(generic_synonyms_or_hypernyms[entity_type])}"
                # Memorizza l'pseudonimo per l'entità per evitare duplicazioni
                pseudonyms[entity] = pseudonym
                return pseudonym
        else:
            return entity  # Restituisce l'entità originale se non è di un tipo gestito

    # Funzione per anonimizzare l'entità di tipo MISC
    def anonymize_misc(entity):
        if len(entity) > 4:
            return f"{entity[:3]}{'*' * (len(entity) - 4)}{entity[-1]}"
        else:
            return entity

    # Ricostruisce il testo originale e applica l'anonimizzazione
    anonymized_text = ""  # Testo anonimizzato inizialmente vuoto
    current_tag = None    # Tag attualmente in esame (PER, ORG, LOC, MISC)
    buffer = []           # Buffer per accumulare le parole di un'entità con lo stesso tag

    # Itera attraverso tutte le righe del dataframe con il tagging BIO
    for index, row in ner_df.iterrows():
        word = row['WORD']  # Parola corrente
        tag = row['TAG']    # Tag corrente della parola

        # Gestione del tag iniziale di un'entità (B-TYPE)
        if tag.startswith('B-'):
            # Se c'è già del testo nel buffer, lo processa prima di iniziare una nuova entità
            if buffer:
                if current_tag in ['PER', 'ORG', 'LOC']:
                    # Ottiene o genera l'pseudonimo per l'entità nel buffer e lo aggiunge al testo anonimizzato
                    pseudonym = get_pseudonym(current_tag, ' '.join(buffer))
                    anonymized_text += pseudonym + ' '
                elif current_tag == 'MISC':
                    # Anonimizza l'entità di tipo MISC nel buffer e la aggiunge al testo anonimizzato
                    anonymized_text += anonymize_misc(' '.join(buffer)) + ' '
                else:
                    # Aggiunge le parole nel buffer direttamente al testo anonimizzato
                    anonymized_text += ' '.join(buffer) + ' '
                buffer = []  # Svuota il buffer dopo aver processato l'entità corrente

            current_tag = tag[2:]  # Estrae il tipo di entità dal tag (es. PER, ORG, LOC)
            buffer.append(word)   # Aggiunge la parola corrente al buffer delle parole dell'entità

        # Gestione delle parole interne di un'entità (I-TYPE)
        elif tag.startswith('I-') and current_tag == tag[2:]:
            buffer.append(word)  # Aggiunge la parola corrente al buffer delle parole dell'entità

        # Gestione delle parole che non fanno parte di un'entità (O-TYPE)
        else:
            # Se ci sono parole nel buffer, le processa prima di aggiungere la parola corrente
            if buffer:
                if current_tag in ['PER', 'ORG', 'LOC']:
                    # Ottiene o genera l'pseudonimo per l'entità nel buffer e lo aggiunge al testo anonimizzato
                    pseudonym = get_pseudonym(current_tag, ' '.join(buffer))
                    anonymized_text += pseudonym + ' '
                elif current_tag == 'MISC':
                    # Anonimizza l'entità di tipo MISC nel buffer e la aggiunge al testo anonimizzato
                    anonymized_text += anonymize_misc(' '.join(buffer)) + ' '
                else:
                    # Aggiunge le parole nel buffer direttamente al testo anonimizzato
                    anonymized_text += ' '.join(buffer) + ' '
                buffer = []       # Svuota il buffer dopo aver processato l'entità corrente
                current_tag = None  # Resetta il tag corrente dopo aver processato l'entità corrente

            # Gestisce la punteggiatura senza aggiungere spazi prima
            if word in [".", ",", "!", "?"]:
                anonymized_text = anonymized_text.rstrip() + word + ' '
            else:
                anonymized_text += word + ' '  # Aggiunge la parola corrente al testo anonimizzato

    # Processa l'eventuale contenuto rimanente nel buffer alla fine del dataframe
    if buffer:
        if current_tag in ['PER', 'ORG', 'LOC']:
            pseudonym = get_pseudonym(current_tag, ' '.join(buffer))
            anonymized_text += pseudonym + ' '
        elif current_tag == 'MISC':
            anonymized_text += anonymize_misc(' '.join(buffer)) + ' '
        else:
            anonymized_text += ' '.join(buffer) + ' '

    # Rimuove eventuali spazi finali dal testo anonimizzato
    anonymized_text = anonymized_text.strip()

    return anonymized_text


In [41]:
ner_df = read_tagging("viterbi_tag")
golden_df = read_tagging("golden_tag")

vit_anonymized_document = anonymize_document(ner_df)
golden_anonymized_document = anonymize_document(golden_df)
print('---HMM Taggin Anonimizzation------')
print(vit_anonymized_document)
print('---Golden Taggin Anonimizzation------')
print(golden_anonymized_document)

---HMM Taggin Anonimizzation------
Si stabilì ad [Loc1] per la sua ammirazione nei confronti della letteratura tedesca ( aveva imparato la lingua in carcere ), specialmente per i romantici come [Per1] individuo. Anche i veicoli interplanetari utilizzano principalmente i razzi chimici, anche se alcuni hanno usato sperimentalmente con successo i nuovi propulsori ionici ( come ad esempio il satellite dell' [Org1] organizzazione SMART-1 ). Dal 1866 al 1869 egli frequentò il liceo di recente costruzione a [Loc2], e nel 1870 egli passò gli esami sulle lingue classiche necessario per l' ammissione all' università. Una******a militare venne costruita attraverso le colline che vanno da [Loc3] fino a [Loc4]. Raggiunge la popolarità nel 1988 nel varietà televisivo " Cocco " di [Org2] azienda con il personaggio dell' automobilista " incazzato come una bestia! ". Nel 2009, 2010, 2011, 2012, 2013, 2014 e 2016 continua a partecipare a " Zelig " TV in onda su [Org3] azienda. Nel 2013 è nel " cast " di